In [2]:
import requests
from datetime import datetime, timedelta

In [ ]:
# --- CONFIGURATION ---
GITHUB_TOKEN = "<Your-GitHub-Token-For-Read>"
TARGET_OWNER = "PowerShell"
TARGET_REPO = "PowerShell"
TARGET_REPO_FULL = f"{TARGET_OWNER}/{TARGET_REPO}"
MAINTAINERS = ["daxian-dbw"] # Add your usernames here
DAYS_BACK = 30  # How far back to look?

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

In [ ]:
# --- GRAPHQL QUERY WITH PAGINATION ---
# This query is for getting the issue or pull request comments left by a github user
QUERY_PAGINATED = """
## The default order for 'issueComments' in the GitHub GraphQL API is typically 
## sorted by creation time in ascending order (oldest comments first).
## So, I need to use `last` and `before` to get the most recent records first.
query($username: String!, $count: Int = 100, $before: String) {
  user(login: $username) {
    issueComments(last: $count, before: $before) {
      pageInfo {
        hasPreviousPage
        startCursor
      }
      nodes {
        publishedAt
        url
        issue {
          author {
            login
          }
          repository {
            nameWithOwner
          }
          publishedAt
          number
          title
        }
        pullRequest {
          merged
        }
      }
    }
  }
}
"""

In [6]:
def run_query_paginated(username, max_comments=None):
    """
    Fetch issue comments for a user with pagination support.
    
    Args:
        username: GitHub username
        max_comments: Maximum number of comments to fetch (None = all available)
    
    Returns:
        List of comments filtered for TARGET_REPO_FULL
    """
    url = 'https://api.github.com/graphql'
    headers = {'Authorization': f'Bearer {GITHUB_TOKEN}'}
    
    all_comments = []
    has_more = True
    cursor = None
    
    while has_more:
        variables = {
            'username': username,
            'count': 100
        }
        
        if cursor:
            variables['before'] = cursor
        
        response = requests.post(url, json={'query': QUERY_PAGINATED, 'variables': variables}, headers=headers)
        
        if response.status_code != 200:
            raise Exception(f"Query failed: {response.status_code} {response.text}")
        
        result = response.json()
        
        if 'data' not in result or 'user' not in result['data'] or not result['data']['user']:
            break
        
        issue_comments = result['data']['user']['issueComments']
        comments = issue_comments['nodes']
        page_info = issue_comments['pageInfo']
        
        # Filter for the target repo
        filtered = [c for c in comments if c['issue']['repository']['nameWithOwner'] == TARGET_REPO_FULL]
        all_comments.extend(filtered)
        
        # Check if we should continue
        has_more = page_info['hasPreviousPage']
        cursor = page_info['startCursor']
        
        # Stop if we've reached the max_comments limit
        if max_comments and len(all_comments) >= max_comments:
            all_comments = all_comments[:max_comments]
            break
    
    return all_comments

In [37]:
run_query_paginated('daxian-dbw', 50)

[{'publishedAt': '2025-09-30T17:01:27Z',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26086#issuecomment-3353065167',
  'issue': {'author': {'login': 'daxian-dbw'},
   'repository': {'nameWithOwner': 'PowerShell/PowerShell'},
   'publishedAt': '2025-09-23T20:12:42Z',
   'number': 26086,
   'title': 'Allow opt-out of the named-pipe listener using an environment variable'},
  'pullRequest': {'merged': True}},
 {'publishedAt': '2025-09-30T17:31:22Z',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26086#issuecomment-3353175169',
  'issue': {'author': {'login': 'daxian-dbw'},
   'repository': {'nameWithOwner': 'PowerShell/PowerShell'},
   'publishedAt': '2025-09-23T20:12:42Z',
   'number': 26086,
   'title': 'Allow opt-out of the named-pipe listener using an environment variable'},
  'pullRequest': {'merged': True}},
 {'publishedAt': '2025-09-30T18:31:35Z',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26091#issuecomment-3353344177',
  'issue': {'author': {'log

In [ ]:
# --- GRAPHQL QUERY FOR PR REVIEWS WITH PAGINATION ---
# This query is for get the PR reviews done by a github user.
PR_REVIEW_QUERY_PAGINATED = """
## The pullRequestReviewContributions field is a connection that orders contributions 
## by default in descending chronological order (most recent contributions first).
## So, I need to use `first` and `after` to get the most recent records first.
query($username: String!, $count: Int = 100, $after: String) {
  user(login: $username) {
    contributionsCollection {
      pullRequestReviewContributions(first: $count, after: $after) {
        pageInfo {
          hasPreviousPage
          startCursor
        }
        nodes {
          occurredAt
          pullRequest {
            author {
              login
            }
            publishedAt
            number
            title
            merged
          }
          pullRequestReview {
            url
            state
          }
          repository {
            nameWithOwner
          }
        }
      }
    }
  }
}
"""

In [53]:
def run_pr_review_query_paginated(username, max_reviews=None):
    """
    Fetch pull request reviews for a user with pagination support.
    
    Args:
        username: GitHub username
        max_reviews: Maximum number of reviews to fetch (None = all available)
    
    Returns:
        List of reviews filtered for TARGET_REPO_FULL
    """
    url = 'https://api.github.com/graphql'
    headers = {'Authorization': f'Bearer {GITHUB_TOKEN}'}
    
    all_reviews = []
    has_more = True
    cursor = None
    
    while has_more:
        variables = {
            'username': username,
            'count': 100
        }
        
        if cursor:
            variables['after'] = cursor
        
        response = requests.post(url, json={'query': PR_REVIEW_QUERY_PAGINATED, 'variables': variables}, headers=headers)
        
        if response.status_code != 200:
            raise Exception(f"Query failed: {response.status_code} {response.text}")
        
        result = response.json()
        
        if 'data' not in result or 'user' not in result['data'] or not result['data']['user']:
            break
        
        contributions = result['data']['user']['contributionsCollection']['pullRequestReviewContributions']
        reviews = contributions['nodes']
        page_info = contributions['pageInfo']
        
        # Filter for the target repo
        filtered = [r for r in reviews if r['repository']['nameWithOwner'] == TARGET_REPO_FULL]
        all_reviews.extend(filtered)
        
        # Check if we should continue
        has_more = page_info['hasPreviousPage']
        cursor = page_info['startCursor']
        
        # Stop if we've reached the max_reviews limit
        if max_reviews and len(all_reviews) >= max_reviews:
            all_reviews = all_reviews[:max_reviews]
            break
    
    return all_reviews

In [56]:
run_pr_review_query_paginated('daxian-dbw', 30)

[{'occurredAt': '2025-12-08T23:21:41Z',
  'pullRequest': {'author': {'login': 'adityapatwardhan'},
   'publishedAt': '2025-12-08T23:19:31Z',
   'number': 26590,
   'title': '[release/v7.6] Update Microsoft.PowerShell.PSResourceGet to v1.2.0-preview5',
   'merged': True},
  'pullRequestReview': {'url': 'https://github.com/PowerShell/PowerShell/pull/26590#pullrequestreview-3554509627',
   'state': 'APPROVED'},
  'repository': {'nameWithOwner': 'PowerShell/PowerShell'}},
 {'occurredAt': '2025-12-08T20:15:23Z',
  'pullRequest': {'author': {'login': 'dkaszews'},
   'publishedAt': '2022-08-31T17:28:25Z',
   'number': 18003,
   'title': 'Make prompt colors configurable',
   'merged': False},
  'pullRequestReview': {'url': 'https://github.com/PowerShell/PowerShell/pull/18003#pullrequestreview-3553807109',
   'state': 'CHANGES_REQUESTED'},
  'repository': {'nameWithOwner': 'PowerShell/PowerShell'}},
 {'occurredAt': '2025-12-05T23:36:17Z',
  'pullRequest': {'author': {'login': 'TravisEz13'},
   

In [ ]:
# --- GRAPHQL QUERY FOR ISSUE AND PR ACTIVITY ---
# This query is for getting LabelEvent and CloseEvent on issues and CloseEvent and MergeEvent on PRs.
REPO_ACTIVITY_QUERY = """
query(
  $owner: String!,
  $repo: String!,
  $since: DateTime!,
  $issuesPageSize: Int = 50,
  $issuesCursor: String,
  $prsPageSize: Int = 50,
  $prsCursor: String,
  $includeIssues: Boolean! = true,
  $includePRs: Boolean! = true
) {
  repository(owner: $owner, name: $repo) {
    issues(
      first: $issuesPageSize,
      after: $issuesCursor,
      orderBy: {field: UPDATED_AT, direction: DESC},
      filterBy: {since: $since}
    ) @include(if: $includeIssues) {
      pageInfo {
        hasNextPage
        endCursor
      }
      nodes {
        number
        title
        url
        updatedAt
        timelineItems(last: 50, itemTypes: [LABELED_EVENT, CLOSED_EVENT]) {
          nodes {
            __typename
            ... on LabeledEvent {
              createdAt
              actor { login }
              label { name }
            }
            ... on ClosedEvent {
              createdAt
              actor { login }
            }
          }
        }
      }
    }
    pullRequests(
      first: $prsPageSize,
      after: $prsCursor,
      orderBy: {field: UPDATED_AT, direction: DESC},
      states: [OPEN, CLOSED, MERGED]
    ) @include(if: $includePRs) {
      pageInfo {
        hasNextPage
        endCursor
      }
      nodes {
        number
        title
        url
        state
        updatedAt
        closedAt
        mergedAt
        timelineItems(last: 50, itemTypes: [CLOSED_EVENT, MERGED_EVENT]) {
          nodes {
            __typename
            ... on ClosedEvent {
              createdAt
              actor { login }
            }
            ... on MergedEvent {
              createdAt
              actor { login }
            }
          }
        }
      }
    }
  }
}
"""


def _post_graphql(query, variables):
    """Execute a GitHub GraphQL query and return the data payload."""
    url = "https://api.github.com/graphql"
    headers = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
    response = requests.post(url, json={"query": query, "variables": variables}, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Query failed: {response.status_code} {response.text}")
    payload = response.json()
    if "errors" in payload:
        raise Exception(payload["errors"])
    return payload.get("data", {})


def _since_datetime(days_back):
    """Return a UTC datetime that is N days in the past."""
    return datetime.utcnow() - timedelta(days=days_back)


def _fetch_recent_issues(owner, repo, since_dt, page_size=50):
    """Pull issues updated since `since_dt` (descending) until we fall below the window."""
    cursor = None
    issues = []
    iso_since = since_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    while True:
        variables = {
            "owner": owner,
            "repo": repo,
            "since": iso_since,
            "issuesPageSize": page_size,
            "issuesCursor": cursor,
            "includeIssues": True,
            "includePRs": False,
        }
        data = _post_graphql(REPO_ACTIVITY_QUERY, variables)
        repo_data = data.get("repository")
        connection = repo_data.get("issues") if repo_data else None
        if not connection:
            break
        for issue in connection["nodes"]:
            if parse_date(issue["updatedAt"]) < since_dt:
                return issues
            issues.append(issue)
        if not connection["pageInfo"]["hasNextPage"]:
            break
        cursor = connection["pageInfo"]["endCursor"]
    return issues


def _fetch_recent_prs(owner, repo, since_dt, page_size=50):
    """Pull PRs updated since `since_dt` (descending) until we fall below the window."""
    cursor = None
    prs = []
    iso_since = since_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    while True:
        variables = {
            "owner": owner,
            "repo": repo,
            "since": iso_since,
            "prsPageSize": page_size,
            "prsCursor": cursor,
            "includeIssues": False,
            "includePRs": True,
        }
        data = _post_graphql(REPO_ACTIVITY_QUERY, variables)
        repo_data = data.get("repository")
        connection = repo_data.get("pullRequests") if repo_data else None
        if not connection:
            break
        for pr in connection["nodes"]:
            if parse_date(pr["updatedAt"]) < since_dt:
                return prs
            prs.append(pr)
        if not connection["pageInfo"]["hasNextPage"]:
            break
        cursor = connection["pageInfo"]["endCursor"]
    return prs


def issues_labeled_resolution_by(actor_login, days_back=DAYS_BACK, owner=TARGET_OWNER, repo=TARGET_REPO):
    """Return issues updated in the window that received a Resolution-* label from `actor_login`."""
    since_dt = _since_datetime(days_back)
    actor = actor_login.lower()
    matches = []
    for issue in _fetch_recent_issues(owner, repo, since_dt):
        for event in issue.get("timelineItems", {}).get("nodes", []):
            if event.get("__typename") != "LabeledEvent":
                continue
            label = (event.get("label") or {}).get("name") or ""
            if not label.startswith("Resolution-"):
                continue
            event_actor = (event.get("actor") or {}).get("login")
            if not event_actor or event_actor.lower() != actor:
                continue
            if parse_date(event["createdAt"]) < since_dt:
                continue
            matches.append(
                {
                    "number": issue["number"],
                    "title": issue["title"],
                    "url": issue["url"],
                    "label": label,
                    "labeledAt": event["createdAt"],
                }
            )
            break
    return matches


def issues_closed_by(actor_login, days_back=DAYS_BACK, owner=TARGET_OWNER, repo=TARGET_REPO):
    """Return issues updated in the window that were closed by `actor_login`."""
    since_dt = _since_datetime(days_back)
    actor = actor_login.lower()
    matches = []
    for issue in _fetch_recent_issues(owner, repo, since_dt):
        for event in issue.get("timelineItems", {}).get("nodes", []):
            if event.get("__typename") != "ClosedEvent":
                continue
            event_actor = (event.get("actor") or {}).get("login")
            if not event_actor or event_actor.lower() != actor:
                continue
            if parse_date(event["createdAt"]) < since_dt:
                continue
            matches.append(
                {
                    "number": issue["number"],
                    "title": issue["title"],
                    "url": issue["url"],
                    "closedAt": event["createdAt"],
                }
            )
            break
    return matches


def prs_closed_or_merged_by(actor_login, days_back=DAYS_BACK, owner=TARGET_OWNER, repo=TARGET_REPO):
    """Return PRs updated in the window that were closed or merged by `actor_login`."""
    since_dt = _since_datetime(days_back)
    actor = actor_login.lower()
    matches = []
    for pr in _fetch_recent_prs(owner, repo, since_dt):
        for event in pr.get("timelineItems", {}).get("nodes", []):
            typename = event.get("__typename")
            if typename not in {"ClosedEvent", "MergedEvent"}:
                continue
            event_actor = (event.get("actor") or {}).get("login")
            if not event_actor or event_actor.lower() != actor:
                continue
            if parse_date(event["createdAt"]) < since_dt:
                continue
            matches.append(
                {
                    "number": pr["number"],
                    "title": pr["title"],
                    "url": pr["url"],
                    "action": "merged" if typename == "MergedEvent" else "closed",
                    "occurredAt": event["createdAt"],
                }
            )
            break
    return matches

In [11]:
# Validate: fetch issues labeled Resolution-* by daxian-dbw
resolution_issues = issues_labeled_resolution_by('daxian-dbw')
print(
    f"Found {len(resolution_issues)} issues labeled Resolution-* by daxian-dbw in the past {DAYS_BACK} days."
)
for issue in resolution_issues:
    print(
        f"#{issue['number']} • {issue['title']} • {issue['label']} • {issue['labeledAt']}\n  {issue['url']}"
    )
resolution_issues

Found 9 issues labeled Resolution-* by daxian-dbw in the past 30 days.
#26682 • PowerShell SDK (in-proc) + Set-ExecutionPolicy AllSigned -Scope Process causes AuthorizationManager check failed when loading PackageManagement.format.ps1xml in a new runspace • Resolution-Answered • 2026-01-12T20:15:10Z
  https://github.com/PowerShell/PowerShell/issues/26682
#26672 • Functions return collections as array regardless of type • Resolution-Answered • 2026-01-09T18:00:33Z
  https://github.com/PowerShell/PowerShell/issues/26672
#26473 • Module manifests are not platform-agnostic for path parsing on install • Resolution-Answered • 2026-01-07T19:55:13Z
  https://github.com/PowerShell/PowerShell/issues/26473
#26674 • GIN BIOSserialNUMBER • Resolution-Answered • 2026-01-09T18:02:48Z
  https://github.com/PowerShell/PowerShell/issues/26674
#26663 • Unable to connect with ExchangeOnline for unattended scripts. Getting failed only when scheduled. • Resolution-External • 2026-01-07T22:32:35Z
  https://gi

[{'number': 26682,
  'title': 'PowerShell SDK (in-proc) + Set-ExecutionPolicy AllSigned -Scope Process causes AuthorizationManager check failed when loading PackageManagement.format.ps1xml in a new runspace',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26682',
  'label': 'Resolution-Answered',
  'labeledAt': '2026-01-12T20:15:10Z'},
 {'number': 26672,
  'title': 'Functions return collections as array regardless of type',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26672',
  'label': 'Resolution-Answered',
  'labeledAt': '2026-01-09T18:00:33Z'},
 {'number': 26473,
  'title': 'Module manifests are not platform-agnostic for path parsing on install',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26473',
  'label': 'Resolution-Answered',
  'labeledAt': '2026-01-07T19:55:13Z'},
 {'number': 26674,
  'title': 'GIN BIOSserialNUMBER',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26674',
  'label': 'Resolution-Answered',
  'labeledAt': '2026-01

In [12]:
# Validate: fetch issues closed by daxian-dbw
closed_issues = issues_closed_by('daxian-dbw')
print(
    f"Found {len(closed_issues)} issues closed by daxian-dbw in the past {DAYS_BACK} days."
)
for issue in closed_issues:
    print(
        f"#{issue['number']} • {issue['title']} • closed {issue['closedAt']}\n  {issue['url']}"
    )
closed_issues

Found 4 issues closed by daxian-dbw in the past 30 days.
#26630 • PowerShell 7.5 ASP.NET Core 8 `Microsoft.Extensions.*` assemblies while running on .NET 9 • closed 2026-01-12T18:49:36Z
  https://github.com/PowerShell/PowerShell/issues/26630
#26543 • Format-* cmdlets throw NullReferenceException when -Property is an empty array • closed 2026-01-12T18:47:43Z
  https://github.com/PowerShell/PowerShell/issues/26543
#26665 • Powershell SDK - Fatal ExecutionEngineException when executing Show-Command on .NET 10 • closed 2026-01-12T18:44:56Z
  https://github.com/PowerShell/PowerShell/issues/26665
#26620 • `[System.Management.Automation.CompletionResult]::new` considered empty string as null • closed 2026-01-12T18:45:43Z
  https://github.com/PowerShell/PowerShell/issues/26620


[{'number': 26630,
  'title': 'PowerShell 7.5 ASP.NET Core 8 `Microsoft.Extensions.*` assemblies while running on .NET 9',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26630',
  'closedAt': '2026-01-12T18:49:36Z'},
 {'number': 26543,
  'title': 'Format-* cmdlets throw NullReferenceException when -Property is an empty array',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26543',
  'closedAt': '2026-01-12T18:47:43Z'},
 {'number': 26665,
  'title': 'Powershell SDK - Fatal ExecutionEngineException when executing Show-Command on .NET 10',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26665',
  'closedAt': '2026-01-12T18:44:56Z'},
 {'number': 26620,
  'title': '`[System.Management.Automation.CompletionResult]::new` considered empty string as null',
  'url': 'https://github.com/PowerShell/PowerShell/issues/26620',
  'closedAt': '2026-01-12T18:45:43Z'}]

In [9]:
# Validate: fetch PRs closed or merged by daxian-dbw
prs_closed_or_merged = prs_closed_or_merged_by('daxian-dbw')
closed_prs = [pr for pr in prs_closed_or_merged if pr['action'] == 'closed']
print(
    f"Found {len(closed_prs)} PRs closed by daxian-dbw in the past {DAYS_BACK} days."
)
for pr in closed_prs:
    print(
        f"#{pr['number']} • {pr['title']} • closed {pr['occurredAt']}\n  {pr['url']}"
    )
closed_prs

Found 4 PRs closed by daxian-dbw in the past 30 days.
#26516 • Update outdated package references • closed 2025-12-15T18:35:23Z
  https://github.com/PowerShell/PowerShell/pull/26516
#26546 • Update outdated package references • closed 2025-12-15T18:35:01Z
  https://github.com/PowerShell/PowerShell/pull/26546
#26582 • Update outdated package references • closed 2025-12-15T18:34:45Z
  https://github.com/PowerShell/PowerShell/pull/26582
#26613 • Update outdated package references • closed 2025-12-15T18:34:09Z
  https://github.com/PowerShell/PowerShell/pull/26613


[{'number': 26516,
  'title': 'Update outdated package references',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26516',
  'action': 'closed',
  'occurredAt': '2025-12-15T18:35:23Z'},
 {'number': 26546,
  'title': 'Update outdated package references',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26546',
  'action': 'closed',
  'occurredAt': '2025-12-15T18:35:01Z'},
 {'number': 26582,
  'title': 'Update outdated package references',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26582',
  'action': 'closed',
  'occurredAt': '2025-12-15T18:34:45Z'},
 {'number': 26613,
  'title': 'Update outdated package references',
  'url': 'https://github.com/PowerShell/PowerShell/pull/26613',
  'action': 'closed',
  'occurredAt': '2025-12-15T18:34:09Z'}]